In [1]:
import numpy as np
import pandas as pd

Main changes:
* Added period 0
* originfips=prev non-null fips &rarr; originfips=prev address_observation fips
  - Causes different moves to be filtered out
* Writes all moves to allmoves.csv
* period=0 &rarr; period=''
* originfips="first record" &rarr; originfips=-1
* prev_effdate=nan &rarr; prev_effdate=-1
* Removed fips leading zeros
* Removed decimal points

Performance:

Process Data-allstates-Three Periods.ipynb lowest of three runs: 0.9s\
&darr;\
Process Data-allstates-Three Periods v2.ipynb lowest of three runs: 0.2s

In [ ]:
# TODO: See what floats can be converted to ints

In [31]:
### Create data/allmoves.csv ###

# TODO: Use read_csv(chunksize=)/dask

# Options
z4types = ('S', 'H', 'R')

# Settings
usecols_all_states = ['pid', 'idate', 'odate', 'z4type', 'effdate']
for i in range(2, 11):
    usecols_all_states.append(f'z4type{i}')
    usecols_all_states.append(f'effdate{i}')
for i in range(1, 11):
    usecols_all_states.append(f'fips{i}')

dtype_all_states = {
    'pid': 'object',
    'idate': 'float',
    'odate': 'float',
    'z4type': 'category',
    'effdate': 'float',
    'fips1': 'float',
}
for i in range(2, 11):
    dtype_all_states[f'z4type{i}'] = 'category'
    dtype_all_states[f'effdate{i}'] = 'float'
    dtype_all_states[f'fips{i}'] = 'float'

na_values_all_states = "Not in California"

list_dict_col_names = [
    {
        'pid': 'pid',
        'idate': 'idate',
        'odate': 'odate',
        'z4type': 'z4type',
        'effdate': 'effdate',
        'fips1': 'fips',
        'seentime': 'seentime',
    }
]
for i in range(2, 11):
    list_dict_col_names.append(
        {
            'pid': 'pid',
            'idate': 'idate',
            'odate': 'odate',
            f'z4type{i}': 'z4type',
            f'effdate{i}': 'effdate',
            f'fips{i}': 'fips',
            'seentime': 'seentime',
        }
    )

# Import columns of interest
df_all_states = pd.read_csv(
    "data/all_states.csv", 
    usecols=usecols_all_states,
    dtype=dtype_all_states,
    na_values=na_values_all_states
)

# Calculate seentime
df_all_states['seentime'] = df_all_states['odate'] - df_all_states['idate']

# Split all_states into individual moves
list_df_all_moves = [
    df_all_states[list_dict_col_names[i].keys()].rename(
        columns=list_dict_col_names[i]
    ) for i in range(10)
]

# Add address_observation column
for i in range(len(list_df_all_moves)):
    list_df_all_moves[i].insert(0, 'address_observation', i + 1)

# Interleave list_df_all_moves
df_all_moves = (
    pd.concat(list_df_all_moves)
        .sort_index(kind='stable')
        .reset_index(drop=True)
)

# Add previous values
df_all_moves[['originfips', 'prev_effdate']] = df_all_moves.groupby('pid')[
    ['fips', 'effdate']
].shift(fill_value=-1)

# Filter df_all_moves
df_all_moves = df_all_moves.dropna(subset=['fips', 'originfips'])
df_all_moves = df_all_moves[df_all_moves['z4type'].isin(z4types)]

# Categorize periods
condList_period = [
    df_all_moves['effdate'].between(201301, 201801, inclusive='left'),
    df_all_moves['effdate'].between(200801, 201301, inclusive='left'),
    df_all_moves['effdate'].between(200301, 200801, inclusive='left'),
]
choiceList_period = ['2', '1', '0']
df_all_moves.insert(
    7,
    'period',
    pd.Categorical(np.select(condList_period, choiceList_period, default=''))
)

In [ ]:
# Write data/allmoves.csv
df_all_moves.to_csv("data/allmoves_v2.csv", float_format="%.0f", index=False)

In [8]:
### Load fips gainers and losers ###

# Settings
usecols_fips_tract = ['tractid_fips', 'gainers', 'losers']

dtype_fips_tract = {
    'tractid_fips': 'float',
    'gainers': 'bool',
    'losers': 'bool',
}

# Import fips_tracts_cats.csv
df_fips_tract = pd.read_csv(
    "fips_tracts_cats.csv",
    usecols=usecols_fips_tract,
    dtype=dtype_fips_tract
)

# Get Series of gainers and losers
se_gainers = df_fips_tract[df_fips_tract['gainers']]['tractid_fips']
se_losers = df_fips_tract[df_fips_tract['losers']]['tractid_fips']

In [3]:
# HACK begin

# Settings
arg_all_moves = {
    'usecols': ['period', 'fips', 'originfips'],
    'dtype': {'period': 'category', 'fips': 'float', 'originfips': 'float'},
    'na_values': "nan",
}

# Load allmoves
df_all_moves = pd.read_csv("data/allmoves_v2.csv", **arg_all_moves)

In [4]:
# Extract period 0 fips and originfips
df_period0 = df_all_moves[df_all_moves['period'] == '0'][['fips', 'originfips']]
df_period1 = df_all_moves[df_all_moves['period'] == '1'][['fips', 'originfips']]
df_period2 = df_all_moves[df_all_moves['period'] == '2'][['fips', 'originfips']]

# end HACK

Main changes:
* Remove leading zeros

In [11]:
### Create gaintotals and losstotals ###

# TODO: Test efficiency

# Count fips
df_gain_total0 = df_period0.groupby('fips').size().reindex(se_gainers)
df_loss_total0 = df_period0.groupby('originfips').size().reindex(se_losers)

df_gain_total1 = df_period1.groupby('fips').size().reindex(se_gainers)
df_loss_total1 = df_period1.groupby('originfips').size().reindex(se_losers)

df_gain_total2 = df_period2.groupby('fips').size().reindex(se_gainers)
df_loss_total2 = df_period2.groupby('originfips').size().reindex(se_losers)

In [12]:
# Settings
arg_to_csv = {
    'na_rep': "0",
    'float_format': "%.0f",
    'header': False,
}

# Write output files
df_gain_total0.to_csv("data/gaintotal_p0.csv", **arg_to_csv)
df_loss_total0.to_csv("data/losstotal_p0.csv", **arg_to_csv)

df_gain_total1.to_csv("data/gaintotal_p1.csv", **arg_to_csv)
df_loss_total1.to_csv("data/losstotal_p1.csv", **arg_to_csv)

df_gain_total2.to_csv("data/gaintotal_p2.csv", **arg_to_csv)
df_loss_total2.to_csv("data/losstotal_p2.csv", **arg_to_csv)

Main changes:
* Index order
* first record &rarr; -1
* Removed leading zeros

In [5]:
### Create matrices ###

# Get all fips in each period
se_indices0 = pd.unique(
    pd.concat([df_period0['originfips'], df_period0['fips']])
)

se_indices1 = pd.unique(
    pd.concat([df_period1['originfips'], df_period1['fips']])
)

se_indices2 = pd.unique(
    pd.concat([df_period2['originfips'], df_period2['fips']])
)

# Create matrix
df_matrix0 = (
    df_period0.groupby(['originfips', 'fips'])
    .size()
    .unstack()
    .reindex(se_indices0, columns=se_indices0)
)

df_matrix1 = (
    df_period1.groupby(['originfips', 'fips'])
    .size()
    .unstack()
    .reindex(se_indices1, columns=se_indices1)
)

df_matrix2 = (
    df_period2.groupby(['originfips', 'fips'])
    .size()
    .unstack()
    .reindex(se_indices2, columns=se_indices2)
)

# Clear top-left cell
df_matrix0.index.name = ''

df_matrix1.index.name = ''

df_matrix2.index.name = ''

In [6]:
# Write matrix
df_matrix0.to_csv("data/matrix_p0.csv", na_rep="0", float_format="%.0f")

df_matrix1.to_csv("data/matrix_p1.csv", na_rep="0", float_format="%.0f")

df_matrix2.to_csv("data/matrix_p2.csv", na_rep="0", float_format="%.0f")

In [2]:
# HACK begin

# Settings
arg_read_csv = {
    'index_col': 0,
    'dtype': 'float',
}

df_matrix0 = pd.read_csv("data/matrix_p0.csv", **arg_read_csv)
df_matrix1 = pd.read_csv("data/matrix_p1.csv", **arg_read_csv)
df_matrix2 = pd.read_csv("data/matrix_p2.csv", **arg_read_csv)

# end HACK

FileNotFoundError: [Errno 2] No such file or directory: 'data/matrix_p2.csv'

Main changes:
* fips='first record' &rarr; fips=-1
* Removed leading zeros

In [11]:
### Create summaries ###

# FIXME: zero out diagonals of matrix first

# Calculate out of high loss
se_high_loss0 = df_matrix1[df_matrix0.index.isin(se_losers)].sum(axis=1)

se_high_loss1 = df_matrix1[df_matrix1.index.isin(se_losers)].sum(axis=1)

se_high_loss2 = df_matrix2[df_matrix2.index.isin(se_losers)].sum(axis=1)

# Calculate into high gain
# df_matrix0_transpose = df_matrix0.transpose()
# df_high_gain0 = df_matrix0_transpose[df_matrix0_transpose.index.isin(se_gainers)].sum(axis=1).to_frame()
df_high_gain0 = df_matrix0[df_matrix0.index.isin(se_gainers)].transpose().sum(axis=1).to_frame(name='count')

# df_matrix1_transpose = df_matrix1.transpose()
# df_high_gain1 = df_matrix1_transpose[df_matrix1_transpose.index.isin(se_gainers)].sum(axis=1).to_frame()
df_high_gain1 = df_matrix1[df_matrix1.index.isin(se_gainers)].transpose().sum(axis=1).to_frame(name='count')

# df_matrix2_transpose = df_matrix2.transpose()
# df_high_gain2 = df_matrix2_transpose[df_matrix2_transpose.index.isin(se_gainers)].sum(axis=1).to_frame()
df_high_gain2 = df_matrix2[df_matrix2.index.isin(se_gainers)].transpose().sum(axis=1).to_frame(name='count')

# Remove index name
df_high_gain0.index.name = ''

df_high_gain1.index.name = ''

df_high_gain2.index.name = ''

# Categorize gain type
choiceList_high_gain = ['gain', 'loss']

condList_high_gain0 = [df_high_gain1.index.isin(se_gainers), df_high_gain1.index.isin(se_losers)]
df_high_gain0['type'] = pd.Categorical(np.select(condList_high_gain0, choiceList_high_gain, default='other'))

condList_high_gain1 = [df_high_gain1.index.isin(se_gainers), df_high_gain1.index.isin(se_losers)]
df_high_gain1['type'] = pd.Categorical(np.select(condList_high_gain1, choiceList_high_gain, default='other'))

condList_high_gain2 = [df_high_gain2.index.isin(se_gainers), df_high_gain2.index.isin(se_losers)]
df_high_gain2['type'] = pd.Categorical(np.select(condList_high_gain2, choiceList_high_gain, default='other'))

# Calculate summary
from_loss0 = df_high_gain0[df_high_gain0.index.isin(se_losers)]['count'].sum()
from_gain0 = df_high_gain0[df_high_gain0.index.isin(se_gainers)]['count'].sum()
from_other0 = df_high_gain0['count'].sum() - from_loss0 - from_gain0

from_loss1 = df_high_gain1[df_high_gain1.index.isin(se_losers)]['count'].sum()
from_gain1 = df_high_gain1[df_high_gain1.index.isin(se_gainers)]['count'].sum()
from_other1 = df_high_gain1['count'].sum() - from_loss1 - from_gain1

from_loss2 = df_high_gain2[df_high_gain2.index.isin(se_losers)]['count'].sum()
from_gain2 = df_high_gain2[df_high_gain2.index.isin(se_gainers)]['count'].sum()
from_other2 = df_high_gain2['count'].sum() - from_loss2 - from_gain2

In [12]:
# Write output files
se_high_loss0.to_csv("data/out_of_high_loss_p0.csv", na_rep="0", float_format="%.0f")
df_high_gain0.to_csv("data/into_high_gain_p0.csv", na_rep="0", float_format="%.0f")
with open("data/into_high_gain_summary_p0.csv", 'w') as f:
    f.write(f"from_loss,{int(from_loss0)}\n")
    f.write(f"from_gain,{int(from_gain0)}\n")
    f.write(f"from_other,{int(from_other0)}\n")

se_high_loss1.to_csv("data/out_of_high_loss_p1.csv", na_rep="0", float_format="%.0f")
df_high_gain1.to_csv("data/into_high_gain_p1.csv", na_rep="0", float_format="%.0f")
with open("data/into_high_gain_summary_p1.csv", 'w') as f:
    f.write(f"from_loss,{int(from_loss1)}\n")
    f.write(f"from_gain,{int(from_gain1)}\n")
    f.write(f"from_other,{int(from_other1)}\n")

se_high_loss2.to_csv("data/out_of_high_loss_p2.csv", na_rep="0", float_format="%.0f")
df_high_gain2.to_csv("data/into_high_gain_p2.csv", na_rep="0", float_format="%.0f")
with open("data/into_high_gain_summary_p2.csv", 'w') as f:
    f.write(f"from_loss,{int(from_loss2)}\n")
    f.write(f"from_gain,{int(from_gain2)}\n")
    f.write(f"from_other,{int(from_other2)}\n")